In [1]:
import numpy as np
import sklearn
import librosa
import tensorflow as tf
from tensorflow import keras
import kapre
from kapre.composed import get_melspectrogram_layer
import os

### Load training data

In [2]:
sample_rate = 44100
mono = True

In [3]:
POSITIVE_DIR = 'matched_samples_positive/'
NEGATIVE_DIR = 'matched_samples_negative/'

matched_samples_positive = [POSITIVE_DIR + name for name in os.listdir(POSITIVE_DIR)]
matched_samples_negative = [NEGATIVE_DIR + name for name in os.listdir(NEGATIVE_DIR)]
all_samples = matched_samples_positive + matched_samples_negative

In [4]:
def standardize_length():
    # For now just shorten to lowest length
    sample_lengths = [len(librosa.load(sample_path)[0]) for sample_path in all_samples]
    return min(sample_lengths)

In [5]:
sample_length = standardize_length()

In [6]:
def load_samples():
    batch_size = len(matched_samples_positive) + len(matched_samples_negative)
    X = np.empty((batch_size, sample_length, 1), dtype=np.float32)
    
    for i, sample_path in enumerate(all_samples):
        sample_wav, sr = librosa.load(sample_path, sr=sample_rate, mono=mono)
        X[i,] = sample_wav[:sample_length].reshape(-1,1)
    
    return X

In [7]:
def load_sample_labels():
    positive_labels = np.ones(len(matched_samples_positive))
    negative_labels = np.zeros(len(matched_samples_negative))
    return np.concatenate((positive_labels, negative_labels), axis=None)

In [8]:
X, y = load_samples(), load_sample_labels()

In [9]:
print('Sample feature shape: ', X.shape)
print('Sample labels shape: ', y.shape)

Sample feature shape:  (311, 2363, 1)
Sample labels shape:  (311,)


In [10]:
# TODO: train_test_split
# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(X, y)

### Simple CNN

In [11]:
input_shape = X[0].shape

In [12]:
spectrogram_layer = get_melspectrogram_layer(input_shape=input_shape,
                             pad_end=True,
                             win_length=400,
                             hop_length=160,
                             sample_rate=sample_rate,
                             return_decibel=True,
                             input_data_format='channels_last',
                             output_data_format='channels_last')

In [13]:
model = keras.models.Sequential([
    spectrogram_layer,
    keras.layers.Conv2D(64, 7, activation="relu", padding="SAME"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="SAME"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="SAME"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="SAME"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="SAME"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid") # binary classification
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 melspectrogram (Sequential)  (None, 15, 128, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 15, 128, 64)       3200      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 64, 64)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 64, 128)        73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 64, 128)        147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 32, 128)       0         
 2D)                                                    

In [15]:
# Compile the model
model.compile('adam', 'binary_crossentropy', metrics=["accuracy"])

In [16]:
model.fit(X, y)

10/10 [==============================] - 3s 171ms/step - loss: 0.8166 - accuracy: 0.6141


In [17]:
model.save("basic_cnn_samples.h5")

In [18]:
X[0].shape
model.predict(X[:10])

array([[0.46367934],
       [0.4443791 ],
       [0.4493065 ],
       [0.4496441 ],
       [0.44529665],
       [0.43239644],
       [0.4670477 ],
       [0.44390786],
       [0.40555426],
       [0.44770366]], dtype=float32)

In [19]:
from keras.models import load_model

kapre_objects = {'STFT':kapre.time_frequency.STFT,
                 'Magnitude':kapre.time_frequency.Magnitude,
                 'ApplyFilterbank':kapre.time_frequency.ApplyFilterbank,
                 'MagnitudeToDecibel':kapre.time_frequency.MagnitudeToDecibel}

loaded_model = load_model('basic_cnn_samples.h5',
                          custom_objects=kapre_objects)

loaded_model.predict(X[:1])

array([[0.46367934]], dtype=float32)

In [22]:
type(X[:10])

with open('test_batch.npy', 'wb') as f:
    np.save(f, X[:10])

In [25]:
with open('test_batch.npy', 'rb') as f:
    a = np.load(f)
loaded_model.predict(a).shape

(10, 1)

### Bring in pre-trained XCeption model

In [ ]:
IMAGE_DIR = 'sample_spec_images/'

In [ ]:
def create_img_filename(filename):
    filename = filename[:-4]
    filename = filename.split('/')[1]
    return IMAGE_DIR + filename + '.png'

In [ ]:
from matplotlib import pyplot as plt
from librosa import display

def save_sample_spectrograms():
    # Save as images for input to Xception model
    for sample_path in all_samples:
        sample, sr = librosa.load(sample_path)
        sample = sample[:sample_length]
        plt.axis('off')
        spectrogram = librosa.feature.melspectrogram(y=sample, sr=sr)
        librosa.display.specshow(librosa.power_to_db(spectrogram, ref=np.max))
        filename = create_img_filename(sample_path)
        plt.savefig(filename)

In [ ]:
save_sample_spectrograms()

In [ ]:
sample_spectrograms = [IMAGE_DIR + name for name in os.listdir(IMAGE_DIR)]

In [ ]:
from PIL import Image

def reformat_images():
    # Reduce size for Xception
    for image_path in sample_spectrograms:
        try:
            image = Image.open(image_path)
            image.thumbnail((250,250))
            image.save(image_path)    
        except IOError:
            print('not an image file: ', image_path)

In [ ]:
reformat_images()

In [ ]:
first_image = Image.open(sample_spectrograms[0])
width, height = first_image.size
channels = 3 # RGB

In [ ]:
def load_sample_images():
    batch_size = len(sample_spectrograms)
    X = np.empty((batch_size, height, width, channels), dtype=np.float32)
    
    for i, image_path in enumerate(sample_spectrograms):
        image_data = np.asarray(image)
        image_data = image_data[:,:,:3] # discard alpha channel
        X[i,] = image_data
    
    return X

In [ ]:
X, y = load_sample_images(), load_sample_labels()

In [ ]:
print('Sample feature shape: ', X.shape)
print('Sample labels shape: ', y.shape)

In [ ]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                 include_top=False,
                                                 input_shape=(height, width, channels))
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = keras.layers.Dense(1, activation="sigmoid")(avg)
model = keras.Model(inputs=base_model.input, outputs=[class_output])
model.compile(loss=["binary_crossentropy", "mse"],
             loss_weights=[0.8, 0.2], optimizer='adam', metrics=["accuracy"])

In [ ]:
model.fit(X, y)

In [ ]:
model.save("xception_samples.h5")